#Plant Disease Detection

#Connect to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
tf.test.gpu_device_name()

#Import libraries

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import tkinter as tk
from tensorflow import keras
from tkinter import filedialog
from PIL import Image, ImageTk
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.utils import to_categorical
from keras.utils import to_categorical

#Read The Data

In [ ]:
dataset_path = "/content/drive/MyDrive/dataset /color"

train_images = keras.utils.image_dataset_from_directory(
    dataset_path,
    image_size=(224, 224),
    batch_size=32,
    seed=123,
    validation_split=0.2,
    subset='training',
    label_mode='categorical'  # Ensure labels are one-hot encoded
)

test_images = keras.utils.image_dataset_from_directory(
    dataset_path,
    image_size=(224, 224),
    batch_size=32,
    seed=123,
    validation_split=0.2,
    subset='validation',
    label_mode='categorical'  # Ensure labels are one-hot encoded
)


In [ ]:
# Function to load image data using cv2.imread
def load_image(file_path):
    full_path = os.path.join(dataset_path, file_path)  # Corrected path
    img = cv2.imread(full_path)
    if img is None:
        print(f"Error loading image: {full_path}")
        return None
    img = cv2.resize(img, (224,224)) # image size
    return img

In [ ]:
def load_dataset(dataset_path, percentage=100):
    image_files = os.listdir(dataset_path)

    all_images = []
    all_labels = []

    for label, class_folder in enumerate(sorted(image_files)):
        class_path = os.path.join(dataset_path, class_folder)
        class_images = [os.path.join(class_path, img) for img in sorted(os.listdir(class_path))]

        # Calculate how many images to load based on the percentage
        num_images_to_load = int(len(class_images) * (percentage / 100.0))

        # Get a subset of images based on the calculated number
        selected_images = class_images[:num_images_to_load]

        for img_path in selected_images:
            if load_image(img_path) is not None:  # Ensure the image is successfully loaded
                all_images.append(img_path)
                all_labels.append(label)

    return all_images, all_labels

# Example usage
all_images, all_labels = load_dataset(dataset_path)

# Example: print the first 10 image paths and their labels
for img_path, label in zip(all_images[:10], all_labels[:10]):
    print(f'{img_path}: {label}')

In [ ]:
classes = train_images.class_names
classes

#Data preprocessing

In [ ]:
# Convert lists to numpy arrays, skipping any None values
loaded_images = [load_image(file_path) for file_path in all_images]
images = np.array([img for img in loaded_images if img is not None])
labels = np.array([label for img, label in zip(loaded_images, all_labels) if img is not None])
print("done convert")

In [ ]:
# Split the labels into training and testing sets
train_labels, test_labels = train_test_split( labels, test_size=0.2, random_state=42)
print("done split")

In [ ]:
# Convert labels to one-hot encoding
#train_labels_onehot = to_categorical(train_labels)
#test_labels_onehot = to_categorical(test_labels)

train_labels_onehot = to_categorical(train_labels, num_classes=38)
test_labels_onehot = to_categorical(test_labels, num_classes=38)
print("done convert ")

In [ ]:
# Use the labels generated by image_dataset_from_directory
train_labels = train_images.map(lambda x, y: y)
test_labels = test_images.map(lambda x, y: y)


#Extracting features

In [ ]:
# Feature extraction model using VGG16 as base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True  # Freeze the weights of the VGG16 base model

#Build CNN model

In [ ]:
# Custom CNN model
model = keras.models.Sequential([
    # Feature extraction layers (VGG16)
    base_model,

    # Add a Global Average Pooling layer
    keras.layers.GlobalAveragePooling2D(),

    # Fully connected layers
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(12, activation='softmax') # Assuming 38 classes for output
    #keras.models.add(Dense(38, activation='softmax'))
])

In [ ]:
# Compile the model
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming train_labels_onehot and test_labels_onehot are one-hot encoded labels
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

In [ ]:
# Example of printing labels and predictions for a batch
for images, labels in train_images.take(1):
    predictions = model.predict(images)
    print("Labels shape:", labels.shape)
    print("Predictions shape:", predictions.shape)


#Train the model

In [ ]:
# Train the model
#history = model.fit(train_images , epochs = 20,validation_data = test_images)

"""# Train the model
history = model.fit(
                    train_images,
                    train_labels,
                    epochs=20,
                    validation_data=(test_images, test_labels))"""

# Train the model using the labels from the dataset
history = model.fit(
    train_images,
    epochs=10,
    validation_data=test_images,
    validation_steps=len(test_images),
)

"""# Train the model
history = model.fit(train_images, train_labels_onehot,
          epochs=20,
          validation_data=(test_images, test_labels_onehot)) """



In [ ]:
accuracy = history.history['accuracy']
loss = history.history['loss']
epochs = range(1,21)

plt.plot(epochs , accuracy , label = 'Acuuracy')
plt.plot(epochs , loss , label = 'loss')
plt.legend()
plt.show()

In [ ]:
model.evaluate(test_images)

#Test model predictions


In [ ]:
def img_to_pred(image):
    image = image.numpy()
    image = tf.expand_dims(image,0)
    return image

In [ ]:
plt.figure(figsize=(18,18))
for images, labels in test_images.take(1) : # take the first patch
  for i in range(1,10):
    plt.subplot(3,3,i)
    plt.imshow(images[i].numpy().astype('uint32'))
    plt.axis('off')
    actual = classes[labels[i]]
    predict =classes[np.argmax( model.predict(img_to_pred(images[i])))]
    plt.title(f"actual : {actual}  \n predicted : {predict} ")

In [ ]:
model.save('Plant_disease.h5')